# setup

## setup dsub

In [ ]:
%%writefile ~/aou_dsub.bash

#!/bin/bash

# This shell function passes reasonable defaults for several dsub parameters, while
# allowing the caller to override any of them. It creates a nice folder structure within
# the workspace bucket for dsub log files.

# --[ Parameters ]--
# any valid dsub parameter flag

#--[ Returns ]--
# the job id of the job created by dsub

#--[ Details ]--
# The first five parameters below should always be those values when running on AoU RWB.

# Feel free to change the values for --user, --regions, --logging, and --image if you like.

# Note that we insert some job data into the logging path.
# https://github.com/DataBiosphere/dsub/blob/main/docs/logging.md#inserting-job-data

function aou_dsub () {

  # Get a shorter username to leave more characters for the job name.
  local DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"

  # For AoU RWB projects network name is "network".
  #local AOU_NETWORK=network
  #local AOU_SUBNETWORK=subnetwork

  dsub \
      --provider google-batch \
      --user-project "${GOOGLE_PROJECT}"\
      --project "${GOOGLE_PROJECT}"\
      --image 'ubuntu:latest' \
      --network "global/networks/network" \
      --subnetwork "regions/us-central1/subnetworks/subnetwork" \
      --service-account "$(gcloud config get-value account)" \
      --use-private-address \
      --user "${DSUB_USER_NAME}" \
      --regions us-central1 \
      --logging "${WORKSPACE_BUCKET}/dsub/logs/{job-name}/{user-id}/$(date +'%Y%m%d/%H%M%S')/{job-id}-{task-id}-{task-attempt}.log" \
      "$@"
}

# compute ancestry specific PCA

## run initial QC

### submit job

In [ ]:
import pandas as pd
import os
bucket = os.getenv('WORKSPACE_BUCKET')

USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.', '-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'inital_qc-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--input-recursive ARRAY': [f"gs://fc-aou-datasets-controlled/v8/microarray/plink" for _ in range(4)],
    '--env ANCESTRY': ['EUR', 'AFR', 'EUR', 'AFR'],
    '--env PHENO': ['TSH'] * 2 + ['Free_T4'] * 2,
    '--env SAMPLE_SIZE' : [12385] * 2 + [4037] * 2,
    '--output-recursive OUTPUT_DIR': [f"{bucket}/PCA/input" for _ in range(4)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep = '\t', index = False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/plink2:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/pca/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --disk-size 360 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'plink2 --bfile ${ARRAY}/arrays \
              --keep $BUCKET/sample_lists/AoU_v8.${PHENO}.GWAS.${ANCESTRY}.n=${SAMPLE_SIZE}.sample_list.txt \
              --geno 0.05 dosage \
              --mind 0.05 dosage \
              --max-alleles 2 \
              --snps-only \
              --make-bed \
              --out ${OUTPUT_DIR}/${ANCESTRY}.${PHENO}.initial_qc'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

## filter based on maf

### check size of files

In [ ]:
!gsutil ls -lh {bucket}/PCA/input/

### submit jobs

In [ ]:
import pandas as pd
import os
bucket = os.getenv('WORKSPACE_BUCKET')

USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.', '-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'maf-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ANCESTRY': ['EUR', 'AFR', 'EUR', 'AFR'],
    '--env PHENO': ['TSH'] * 2 + ['Free_T4'] * 2,
    '--output-recursive OUTPUT_DIR': [f"{bucket}/PCA/input" for _ in range(4)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep = '\t', index = False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/plink2:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/pca/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --disk-size 20 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'plink2 --bfile $BUCKET/PCA/input/${ANCESTRY}.${PHENO}.initial_qc \
              --maf 0.05 \
              --make-bed \
              --out ${OUTPUT_DIR}/${ANCESTRY}.${PHENO}.freq_qc'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

## ld prune

### check initial number of variants

In [ ]:
!gsutil cp {bucket}/PCA/input/*.freq_qc.bim .

In [ ]:
!wc -l *.freq_qc.bim

### check size of files

In [ ]:
!gsutil ls -lh {bucket}/PCA/input/*TSH.freq_qc*

### submit job

In [ ]:
import pandas as pd
import os
bucket = os.getenv('WORKSPACE_BUCKET')

USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.', '-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'ld_prune-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ANCESTRY': ['EUR', 'AFR', 'EUR', 'AFR'],
    '--env PHENO': ['TSH'] * 2 + ['Free_T4'] * 2,
    '--env R2' : ['0.15', '0.065', '0.15', '0.065'],
    '--output-recursive OUTPUT_DIR': [f"{bucket}/PCA/input" for _ in range(4)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep = '\t', index = False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/plink2:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/pca/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --disk-size 5 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'plink2 --bfile $BUCKET/PCA/input/${ANCESTRY}.${PHENO}.freq_qc \
              --autosome \
              --indep-pairwise 150 5 ${R2} \
              --out ${OUTPUT_DIR}/${ANCESTRY}.${PHENO}.ld_pruned'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

### check number of variants after pruning

In [ ]:
!gsutil cp {bucket}/PCA/input/*.ld_pruned.prune.in .

In [ ]:
!wc -l *.ld_pruned.prune.in

## extract pruned variants

### submit job

In [ ]:
import pandas as pd
import os
bucket = os.getenv('WORKSPACE_BUCKET')

USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.', '-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'ld_prune_extract-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ANCESTRY': ['EUR', 'AFR', 'EUR', 'AFR'],
    '--env PHENO': ['TSH'] * 2 + ['Free_T4'] * 2,
    '--output-recursive OUTPUT_DIR': [f"{bucket}/PCA/input" for _ in range(4)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep = '\t', index = False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/plink2:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/pca/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --disk-size 5 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'plink2 --bfile $BUCKET/PCA/input/${ANCESTRY}.${PHENO}.freq_qc \
              --extract $BUCKET/PCA/input/${ANCESTRY}.${PHENO}.ld_pruned.prune.in \
              --make-bed \
              --out ${OUTPUT_DIR}/${ANCESTRY}.${PHENO}.ld_pruned'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

## make convertf input file

### TSH

In [ ]:
%%bash
BUCKET="fc-secure-xxxx"
PHENO='TSH'
for ANCESTRY in EUR AFR
do

echo ${ANCESTRY}
genotypename="/mnt/data/mount/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.ld_pruned.bed"
snpname="/mnt/data/mount/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.ld_pruned.bim"
indivname="/mnt/data/mount/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.ld_pruned.fam"
outputformat="EIGENSTRAT"
genotypeoutname="/mnt/data/output/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.PCA_input.eigenstratgeno"
snpoutname="/mnt/data/output/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.PCA_input.snp"
indivoutname="/mnt/data/output/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.PCA_input.ind" && \

echo "genotypename:    $genotypename" > convertf.${ANCESTRY}.${PHENO}.par
echo "snpname:         $snpname" >> convertf.${ANCESTRY}.${PHENO}.par
echo "indivname:       $indivname" >> convertf.${ANCESTRY}.${PHENO}.par
echo "outputformat:    $outputformat" >> convertf.${ANCESTRY}.${PHENO}.par
echo "genotypeoutname: $genotypeoutname" >> convertf.${ANCESTRY}.${PHENO}.par
echo "snpoutname:      $snpoutname" >> convertf.${ANCESTRY}.${PHENO}.par
echo "indivoutname:    $indivoutname" >> convertf.${ANCESTRY}.${PHENO}.par

cat convertf.${ANCESTRY}.${PHENO}.par
gsutil cp convertf.${ANCESTRY}.${PHENO}.par ${WORKSPACE_BUCKET}/PCA/input/
echo " "
done

### Free T4

In [ ]:
%%bash
BUCKET="fc-secure-xxxx"
PHENO='Free_T4'
for ANCESTRY in EUR AFR
do

echo ${ANCESTRY}
genotypename="/mnt/data/mount/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.ld_pruned.bed"
snpname="/mnt/data/mount/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.ld_pruned.bim"
indivname="/mnt/data/mount/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.ld_pruned.fam"
outputformat="EIGENSTRAT"
genotypeoutname="/mnt/data/output/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.PCA_input.eigenstratgeno"
snpoutname="/mnt/data/output/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.PCA_input.snp"
indivoutname="/mnt/data/output/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.PCA_input.ind" && \

echo "genotypename:    $genotypename" > convertf.${ANCESTRY}.${PHENO}.par
echo "snpname:         $snpname" >> convertf.${ANCESTRY}.${PHENO}.par
echo "indivname:       $indivname" >> convertf.${ANCESTRY}.${PHENO}.par
echo "outputformat:    $outputformat" >> convertf.${ANCESTRY}.${PHENO}.par
echo "genotypeoutname: $genotypeoutname" >> convertf.${ANCESTRY}.${PHENO}.par
echo "snpoutname:      $snpoutname" >> convertf.${ANCESTRY}.${PHENO}.par
echo "indivoutname:    $indivoutname" >> convertf.${ANCESTRY}.${PHENO}.par

cat convertf.${ANCESTRY}.${PHENO}.par
gsutil cp convertf.${ANCESTRY}.${PHENO}.par ${WORKSPACE_BUCKET}/PCA/input/
echo " "
done

## convert plink to eigenstrat format

### submit job

In [ ]:
import pandas as pd
import os
bucket = os.getenv('WORKSPACE_BUCKET')

USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.', '-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'convertf-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ANCESTRY': ['EUR', 'AFR', 'EUR', 'AFR'],
    '--env PHENO': ['TSH'] * 2 + ['Free_T4'] * 2,
    '--output-recursive OUTPUT_DIR': [f"{bucket}/PCA/input/" for _ in range(4)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep = '\t', index = False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/eigenstrat:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/pca/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --disk-size 5 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'convertf -p $BUCKET/PCA/input/convertf.${ANCESTRY}.${PHENO}.par'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

## make smartPCA par file

### TSH

In [ ]:
%%bash
BUCKET="fc-secure-xxxx"
PHENO='TSH'
for ANCESTRY in EUR AFR
do

echo ${ANCESTRY}
genotypename="/mnt/data/mount/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.PCA_input.eigenstratgeno"
snpname="/mnt/data/mount/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.PCA_input.snp"
indivname="/mnt/data/mount/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.PCA_input.ind"
evecoutname="/mnt/data/output/gs/${BUCKET}/PCA/output/${ANCESTRY}.${PHENO}.PCA.eigenvec"
evaloutname="/mnt/data/output/gs/${BUCKET}/PCA/output/${ANCESTRY}.${PHENO}.PCA.eigenval"
numoutevec="20"
numoutlieriter="0"
altnormstyle="NO"
fastmode="NO"
numthreads="8"

echo "genotypename:    $genotypename" > smartPCA.${ANCESTRY}.${PHENO}.par
echo "snpname:         $snpname" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "indivname:       $indivname" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "evecoutname:     $evecoutname" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "evaloutname:     $evaloutname" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "numoutevec:      $numoutevec" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "numoutlieriter:  $numoutlieriter" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "altnormstyle:    $altnormstyle" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "fastmode:        $fastmode" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "numthreads:      $numthreads" >> smartPCA.${ANCESTRY}.${PHENO}.par

cat smartPCA.${ANCESTRY}.${PHENO}.par
gsutil cp smartPCA.${ANCESTRY}.${PHENO}.par ${WORKSPACE_BUCKET}/PCA/input/
echo " "
done

### Free T4

In [ ]:
%%bash
BUCKET="fc-secure-xxxx"
PHENO='Free_T4'
for ANCESTRY in EUR AFR
do

echo ${ANCESTRY}
genotypename="/mnt/data/mount/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.PCA_input.eigenstratgeno"
snpname="/mnt/data/mount/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.PCA_input.snp"
indivname="/mnt/data/mount/gs/${BUCKET}/PCA/input/${ANCESTRY}.${PHENO}.PCA_input.ind"
evecoutname="/mnt/data/output/gs/${BUCKET}/PCA/output/${ANCESTRY}.${PHENO}.PCA.eigenvec"
evaloutname="/mnt/data/output/gs/${BUCKET}/PCA/output/${ANCESTRY}.${PHENO}.PCA.eigenval"
numoutevec="20"
numoutlieriter="0"
altnormstyle="NO"
fastmode="NO"
numthreads="8"

echo "genotypename:    $genotypename" > smartPCA.${ANCESTRY}.${PHENO}.par
echo "snpname:         $snpname" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "indivname:       $indivname" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "evecoutname:     $evecoutname" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "evaloutname:     $evaloutname" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "numoutevec:      $numoutevec" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "numoutlieriter:  $numoutlieriter" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "altnormstyle:    $altnormstyle" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "fastmode:        $fastmode" >> smartPCA.${ANCESTRY}.${PHENO}.par
echo "numthreads:      $numthreads" >> smartPCA.${ANCESTRY}.${PHENO}.par

cat smartPCA.${ANCESTRY}.${PHENO}.par
gsutil cp smartPCA.${ANCESTRY}.${PHENO}.par ${WORKSPACE_BUCKET}/PCA/input/
echo " "
done

## run smartPCA

### submit job

In [ ]:
import pandas as pd
import os
bucket = os.getenv('WORKSPACE_BUCKET')

USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.', '-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'pca-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ANCESTRY': ['EUR', 'AFR', 'EUR', 'AFR'],
    '--env PHENO': ['TSH'] * 2 + ['Free_T4'] * 2,
    '--output EIGENVEC': [f"{bucket}/PCA/output/{x}.{y}.PCA.eigenvec" for x, y in zip(['EUR', 'AFR', 'EUR', 'AFR'], ['TSH', 'TSH', 'Free_T4', 'Free_T4'])],
    '--output EIGENVAL': [f"{bucket}/PCA/output/{x}.{y}.PCA.eigenval" for x, y in zip(['EUR', 'AFR', 'EUR', 'AFR'], ['TSH', 'TSH', 'Free_T4', 'Free_T4'])],
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep = '\t', index = False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/eigenstrat:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/pca/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --disk-size 5 \
  --min-cores 8 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'smartpca -p $BUCKET/PCA/input/smartPCA.${ANCESTRY}.${PHENO}.par'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

## clean smartPCA output

### download output

In [ ]:
!gsutil cp ${WORKSPACE_BUCKET}/PCA/output/*.PCA.* .

### clean eigenvec

### TSH

In [ ]:
%%bash
PHENO='TSH'
for ANCESTRY in EUR AFR
do
awk 'NR>1' ${ANCESTRY}.${PHENO}.PCA.eigenvec | sed 's/^.*0://g' | sed 's/ /,/g' | sed 's/,\+/,/g' | sed 's/,???//g' > ${ANCESTRY}.${PHENO}.PCA_cleaned.eigenvec
gsutil cp ${ANCESTRY}.${PHENO}.PCA_cleaned.eigenvec ${WORKSPACE_BUCKET}/PCA/output/
done

In [ ]:
%%bash
PHENO='Free_T4'
for ANCESTRY in EUR AFR
do
awk 'NR>1' ${ANCESTRY}.${PHENO}.PCA.eigenvec | sed 's/^.*0://g' | sed 's/ /,/g' | sed 's/,\+/,/g' | sed 's/,???//g' > ${ANCESTRY}.${PHENO}.PCA_cleaned.eigenvec
gsutil cp ${ANCESTRY}.${PHENO}.PCA_cleaned.eigenvec ${WORKSPACE_BUCKET}/PCA/output/
done

### clean eigenval

In [ ]:
%%bash
PHENO='TSH'
for ANCESTRY in EUR AFR
do
sed 's/^[[:space:]]*//' ${ANCESTRY}.${PHENO}.PCA.eigenval > ${ANCESTRY}.${PHENO}.PCA_cleaned.eigenval
gsutil cp ${ANCESTRY}.${PHENO}.PCA_cleaned.eigenval ${WORKSPACE_BUCKET}/PCA/output/
done

In [ ]:
%%bash
PHENO='Free_T4'
for ANCESTRY in EUR AFR
do
sed 's/^[[:space:]]*//' ${ANCESTRY}.${PHENO}.PCA.eigenval > ${ANCESTRY}.${PHENO}.PCA_cleaned.eigenval
gsutil cp ${ANCESTRY}.${PHENO}.PCA_cleaned.eigenval ${WORKSPACE_BUCKET}/PCA/output/
done

# GWAS QC

### check size of files

In [ ]:
!gsutil -u $GOOGLE_PROJECT ls -lh gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/acaf_threshold/pgen/acaf_threshold.chr1.*

### submit job

In [ ]:
import pandas as pd
import os
bucket = os.getenv('WORKSPACE_BUCKET')

USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'gwas_qc-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data = {
    '--input PSAM': [f"gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/acaf_threshold/pgen/acaf_threshold.chr{x}.psam" for x in list(range(1, 23)) * 4],
    '--input PVAR': [f"gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/acaf_threshold/pgen/acaf_threshold.chr{x}.pvar" for x in list(range(1, 23)) * 4],
    '--input PGEN': [f"gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/acaf_threshold/pgen/acaf_threshold.chr{x}.pgen" for x in list(range(1, 23)) * 4],
    '--input LONGRANGE_LD': [f"{bucket}/GWAS_QC/input/longrange_LD_hg38.bed" for _ in range(88)],
    '--env ANCESTRY': ['EUR'] * 22 + ['AFR'] * 22 + ['EUR'] * 22 + ['AFR'] * 22,
    '--env PHENO': ['TSH'] * 44 + ['Free_T4'] * 44,
    '--env SAMPLE_SIZE' : [12385] * 44 + [4037] * 44,
    '--env CHR': list(range(1, 23)) * 4,
    '--output-recursive OUTPUT_DIR': [f"{bucket}/GWAS_QC/output/" for _ in range(88)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep = '\t', index = False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/plink2:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/plink_qc/" \
  --disk-size 160 \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'plink2 --psam ${PSAM} \
              --pvar ${PVAR} \
              --pgen ${PGEN} \
              --max-alleles 2 \
              --set-all-var-ids chr@:#:\$r:\$a \
              --keep $BUCKET/sample_lists/AoU_v8.${PHENO}.GWAS.${ANCESTRY}.n=${SAMPLE_SIZE}.sample_list.txt \
              --exclude range ${LONGRANGE_LD} \
              --maf 0.01 \
              --geno 0.01 \
              --hwe 1E-6 \
              --snps-only \
              --make-bed \
              --out ${OUTPUT_DIR}/${ANCESTRY}.${PHENO}.gwas_qc.chr${CHR}'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

# Run PLINK GWAS

### submit job

In [ ]:
import pandas as pd
import os
bucket = os.getenv('WORKSPACE_BUCKET')

USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.', '-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'plink_gwas-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ANCESTRY': ['EUR'] * 22 + ['AFR'] * 22 + ['EUR'] * 22 + ['AFR'] * 22,
    '--env PHENO': ['TSH'] * 44 + ['Free_T4'] * 44,
    '--env SAMPLE_SIZE' : [12385] * 44 + [4037] * 44,
    '--env CHR': list(range(1,23)) * 4,
    '--env PC': ['PC1 PC2 PC3 PC4'] * 22 + ['PC1'] * 22 + ['PC1 PC2 PC3 PC4'] * 22 + ['PC1'] * 22,
    '--env PHENO_NAME': ['INV_NORMAL_TSH'] * 44 + ['FREE_T4'] * 44,
    '--output-recursive OUTPUT_DIR': [f"{bucket}/PLINK_GWAS/chromosome_separated/" for _ in range(88)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep = '\t', index = False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/plink2:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/plink_gwas/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --disk-size 5 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'plink2 --glm hide-covar cols=+a1freq \
                --bfile $BUCKET/GWAS_QC/output/${ANCESTRY}.${PHENO}.gwas_qc.chr${CHR} \
                --pheno $BUCKET/phenotype_files/AoU_v8.${PHENO}.GWAS.${ANCESTRY}.n=${SAMPLE_SIZE}.phenotype_covariates.txt \
                --pheno-name ${PHENO_NAME} \
                --covar $BUCKET/phenotype_files/AoU_v8.${PHENO}.GWAS.${ANCESTRY}.n=${SAMPLE_SIZE}.phenotype_covariates.txt \
                --covar-name AGE_AGE SEX ${PC} \
                --covar-variance-standardize AGE_AGE SEX ${PC} \
                --adjust \
                --out ${OUTPUT_DIR}/AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.chr${CHR}'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

# clean GWAS output

## copy, clean, and recopy

### copy

In [ ]:
!gsutil cp -R ${WORKSPACE_BUCKET}/PLINK_GWAS/chromosome_separated/ .

### split outputs into 2 for export and compress

#### TSH

In [ ]:
%%bash
PHENO='TSH'
PHENO_NAME='INV_NORMAL_TSH'
SAMPLE_SIZE='12385'

for ANCESTRY in EUR AFR
do

echo $ANCESTRY

for chr in $(seq 1 22)
do
awk 'NR>1' "chromosome_separated/AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.chr"${chr}".${PHENO_NAME}.glm.linear" > "chromosome_separated/AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.chr"${chr}".${PHENO_NAME}.glm.linear.no_header"
done


cat chromosome_separated/AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.chr*.${PHENO_NAME}.glm.linear.no_header | awk 'BEGIN {print "#CHROM POS ID REF ALT PROVISIONAL_REF? A1 OMITTED A1_FREQ TEST OBS_CT BETA SE T_STAT P ERRCODE"} {print $0}' | sed 's/ /\t/g' > AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear
cut -f6,9,14,16 --complement AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear > AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export
linecount=$(wc -l AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export | cut -d ' ' -f1)
echo $linecount
half=$(($linecount / 2))
half_plus_one=$(($half +1))
echo $half
head -n $half AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export> AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt1
tail -n $half AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export > AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt2
gzip -f AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt1
gzip -f AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt2
gzip -f AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export

ls -lh AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt1.gz
ls -lh AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt2.gz

zcat AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export | head -n $half_plus_one | tail -n2
zcat AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt1.gz | tail -n1
zcat AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt2.gz | head -n1

echo ' '
done

#### Free T4

In [ ]:
%%bash
PHENO='Free_T4'
PHENO_NAME='FREE_T4'
SAMPLE_SIZE='4037'

for ANCESTRY in EUR AFR
do

echo $ANCESTRY

for chr in $(seq 1 22)
do
awk 'NR>1' "chromosome_separated/AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.chr"${chr}".${PHENO_NAME}.glm.linear" > "chromosome_separated/AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.chr"${chr}".${PHENO_NAME}.glm.linear.no_header"
done


cat chromosome_separated/AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.chr*.${PHENO_NAME}.glm.linear.no_header | awk 'BEGIN {print "#CHROM POS ID REF ALT PROVISIONAL_REF? A1 OMITTED A1_FREQ TEST OBS_CT BETA SE T_STAT P ERRCODE"} {print $0}' | sed 's/ /\t/g' > AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear
cut -f6,9,14,16 --complement AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear > AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export
linecount=$(wc -l AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export | cut -d ' ' -f1)
echo $linecount
half=$(($linecount / 2))
half_plus_one=$(($half +1))
echo $half
head -n $half AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export> AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt1
if (( $linecount % 2 == 1 )); then
echo "odd"
tail -n $half_plus_one AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export > AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt2
else
echo "even"
tail -n $half AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export > AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt2
fi
gzip -f AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt1
gzip -f AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt2
gzip -f AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export

ls -lh AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt1.gz
ls -lh AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt2.gz

zcat AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.gz | head -n $half_plus_one | tail -n2
zcat AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt1.gz | tail -n1
zcat AoU_v8.${PHENO}.${ANCESTRY}.n=${SAMPLE_SIZE}.GWAS_results.all_chr.${PHENO_NAME}.glm.linear.for_export.pt2.gz | head -n1

echo ' '
done

### copy to bucket

In [ ]:
!gsutil -m cp AoU_v8.*GWAS_results.all_chr*.glm.linear ${WORKSPACE_BUCKET}/PLINK_GWAS/chromosome_combined/

In [ ]:
!gsutil -m cp AoU_v8.*.GWAS_results.all_chr.*.glm.linear.for_export* ${WORKSPACE_BUCKET}/PLINK_GWAS/chromosome_combined/